In [2]:
from workflow.scripts.utils import load_CMIP_data, copy_meta_data_CMIP,transelate_aerocom_helper
from pyclim_noresm.general_util_funcs import yearly_avg
import time
import pathlib as pl
import xarray as xr


In [4]:
paths = sorted(snakemake.input.pi_clim_var)

In [8]:
data = load_CMIP_data(paths, data_vars=[snakemake.wildcards.variable])
attrs = copy_meta_data_CMIP(data.attrs)
wildcards = snakemake.wildcards
freq = transelate_aerocom_helper(wildcards)
variable = wildcards.variable

params = snakemake.params
accumlative_vars = params.get('accumalative_vars',None)

In [14]:
with xr.set_options(keep_attrs=True):
    if freq == 'Ayear':
        if 'time_bnds' in data.data_vars:
            data = data.drop_vars('time_bnds')
        if variable in accumlative_vars:
            data=data.resample(time='Y').mean()*365*24*60*60
            data[data.variable_id].attrs['units'] = '{} year-1'.format(' '.join(data[data.variable_id].attrs['units'].split(' ')[:-1]))
            data.attrs['history'] = data.attrs['history'] + f', accumulated over a year'
        else:
            #data[data.variable_id] = yearly_avg(data[data.variable_id])
            data=data.resample(time='Y').mean()
            data.attrs['history'] = data.attrs['history'] + f', annual average'
            data
    elif freq == 'clim':
        t0 = data.time[0].dt.strftime('%Y/%m').values
        t1 = data.time[0].dt.strftime('%Y/%m').values
        data = data.groupby('time.month').mean('time')
        data[data.variable_id].attrs['history'] = data[data.variable_id].attrs.get('history','') + f', clim mean {t0}-{t1}'
        if wildcards.freq=='2010':
            import cftime
            import pandas as pd
            data = data.rename(month='time')
            cftimes = cftime.date2num(pd.date_range('2010-01-31','2010-12-31', freq='M').to_list(),
                                              'days since 2010-01-01', 
                                                  has_year_zero=False, calendar = 'gregorian')
            data = data.assign_coords(time=cftimes)
            data.time.attrs['units'] = 'days since 2010-01-01'
        
    elif freq == 'Amon':
        data = data
    else:
        raise(ValueError(f'{wildcards.freq} is an invalid frequency'))

In [10]:
data = data.assign_attrs(**attrs)


In [11]:
data = data.assign_attrs(variable_id=wildcards.variable)
data.attrs['source'] = ', '.join(snakemake.input)
data.attrs['history'] = f'@{time.ctime()} Generated by: {pl.Path.cwd().parts[-1]}'
data.attrs['frequency'] = snakemake.wildcards.freq
data.to_netcdf(snakemake.output.outpath)